Data Source: [Sales Data from Iowa Class “E” liquor licensees](https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy/about_data) from January 2012 through the end of September 2023. Downloaded November 2023. 

In [1]:
import pandas as pd
import re

In [2]:
liquor_data = pd.read_csv('Iowa_Liquor_Sales.csv')

#drop liquor invoice column
liquor_data = liquor_data.drop(liquor_data.columns[0], axis=1)

/tmp/ipykernel_1058/2407653461.py:1: DtypeWarning: Columns (6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  liquor_data = pd.read_csv('Iowa_Liquor_Sales.csv')


Reading in that data took a long time. A further complication is that a number of columns have mixed data types. Let's do some further investigation. 

In [3]:
print('Length of Dataset: ')
print(len(liquor_data))
print('Number of missing values: ')
num_missing_values = (liquor_data.isnull().sum(axis=1) > 0).sum()
print(num_missing_values)
print('Percentage of missing values: ')
print(num_missing_values/len(liquor_data) * 100)

Length of Dataset: 
27489743
Number of missing values: 
5688292
Percentage of missing values: 
20.692416076789076


As we can see, we have a very large number of rows with missing values, accounting for over 20 percent of the total data. Extensive cleaning will be required before this data will be workable. 

Because the file is so large, it is almost impossible to work with in its entirety. At least on my computer...Therefore, I decided to split it into more digestable chunks based on the year. The data begins in 2012 and ends in 2023, so in the end, I want 12 dataframes corresponding to each year. 

In [4]:
start_year = 2012
end_year = 2023


#initialize row count
total_rows = 0

#dictionary to store dataframes by year
dataframes_by_year = {}

for year in range(start_year, end_year + 1):

    #create a mask to filter for current year
    mask = liquor_data['Date'].str.contains(str(year))

    #instantiate dataframe for current year
    year_data = liquor_data[mask].copy()
    total_rows += len(year_data)

    #define a dataframe for the current year
    year_dataframe_name = f'Iowa_Liquor_Data_{year}'

    #store dataframe in dictionary
    dataframes_by_year[year_dataframe_name] = year_data


# Access the dataframes using the dictionary
print("Dataframes:")
for name, df in dataframes_by_year.items():
    print(f"{name}: {len(df)} rows")

Dataframes:
Iowa_Liquor_Data_2012: 2082059 rows
Iowa_Liquor_Data_2013: 2063763 rows
Iowa_Liquor_Data_2014: 2097796 rows
Iowa_Liquor_Data_2015: 2184483 rows
Iowa_Liquor_Data_2016: 2279893 rows
Iowa_Liquor_Data_2017: 2291276 rows
Iowa_Liquor_Data_2018: 2355558 rows
Iowa_Liquor_Data_2019: 2380345 rows
Iowa_Liquor_Data_2020: 2614365 rows
Iowa_Liquor_Data_2021: 2622712 rows
Iowa_Liquor_Data_2022: 2564565 rows
Iowa_Liquor_Data_2023: 1952928 rows


We kept a variable that tracks the number of rows in each yearly dataframe. Let's make sure we didn't lose any data. 

In [5]:
print('Total rows across all dataframes: ')
print(total_rows)

Total rows across all dataframes: 
27489743


Addressing missing data will be easier if we can separate all of the rows containing missing data into a single dataframe. However, we will need to remove those rows from their original, yearly dataframe to avoid adding duplicates if we add them back in. However, there is a problem. The 2023 data's 'County Number' column is entirely missing, meaning any filtering that removes rows based on the presence of NA values won't be helpful for the year of 2023. 

Additionally, there are large amounts of other rows containing either missing 'County' values or missing 'County Number' values. Let's loop through our yearly dataframes and find out how many rows contain either a missing 'County' value or a missing 'County Number' but not both (XOR). We will also print the counts for each dataframe where this condition is true.

In [6]:
#dictionary to store the missing county data counts for each year
counts_by_year = {}

#iterate through yearly dataframes
for year, df in dataframes_by_year.items():
    #get num rows satisfying missing county condition
    count_condition = (df['County'].isnull() ^ df['County Number'].isnull())
    count = count_condition.sum()  # Count the number of True values

    #store count
    counts_by_year[year] = count

    #output ccount
    print(f"Count of missing county data from {year} : {count}")

Count of missing county data from Iowa_Liquor_Data_2012 : 0
Count of missing county data from Iowa_Liquor_Data_2013 : 0
Count of missing county data from Iowa_Liquor_Data_2014 : 0
Count of missing county data from Iowa_Liquor_Data_2015 : 0
Count of missing county data from Iowa_Liquor_Data_2016 : 0
Count of missing county data from Iowa_Liquor_Data_2017 : 0
Count of missing county data from Iowa_Liquor_Data_2018 : 2
Count of missing county data from Iowa_Liquor_Data_2019 : 0
Count of missing county data from Iowa_Liquor_Data_2020 : 0
Count of missing county data from Iowa_Liquor_Data_2021 : 0
Count of missing county data from Iowa_Liquor_Data_2022 : 1245255
Count of missing county data from Iowa_Liquor_Data_2023 : 1952127


Only a tiny sliver of missing county data occurs in 2018, while the vast majority occurs in either 2022 or 2023. Investigating those 2018 rows reveals that two rows are from non-Iowa cities. We will remove them. 

In [7]:
liquor_2018 = dataframes_by_year['Iowa_Liquor_Data_2018']

# Filter rows where the XOR condition holds for 'County' and 'County Number' columns
xor_condition_rows_2018 = liquor_2018[(liquor_2018['County'].isnull() ^ liquor_2018['County Number'].isnull())]

xor_condition_rows_2018.head()


,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
7983617,01/18/2018,9936,DASH EVENTS LLC,1685 W UINTAH ST. #101,COLORADO SPRINGS,80904.0,POINT (-104.845334 38.848017),NaN,EL PASO,1081200.0,...,78592,ADELAIDES CARAMEL MOCHA MOJO,12,750,10.38,15.57,6,15.57,4.5,1.18
8002556,01/18/2018,9936,DASH EVENTS LLC,1685 W UINTAH ST. #101,COLORADO SPRINGS,80904.0,POINT (-104.845334 38.848017),NaN,EL PASO,1081300.0,...,71412,ADELAIDES COCONUT LIQUEUR,12,750,10.38,15.57,6,15.57,4.5,1.18


In [8]:
#get bad 2018 data
xor_condition_rows_2018 = dataframes_by_year['Iowa_Liquor_Data_2018'][
    (dataframes_by_year['Iowa_Liquor_Data_2018']['County'].isnull() ^
     dataframes_by_year['Iowa_Liquor_Data_2018']['County Number'].isnull())
]

#get indicies of bad rows
rows_to_remove_indices = xor_condition_rows_2018.index

#using indicies, remove the rows from the 2018 DataFrame in dataframes_by_year
dataframes_by_year['Iowa_Liquor_Data_2018'] = dataframes_by_year['Iowa_Liquor_Data_2018'].drop(rows_to_remove_indices)

Now we return to the other missing county data. As we can see, missing county data occurs in only the 2022 and 2023 years, but rows with missing county data represent over 50% of the total rows with missing values. Therefore, it will be crucial to fix this.

In [9]:
total_missing_county_data =  1245255 + 1952127
print(total_missing_county_data / (num_missing_values - 2)) #subtracting 2 since we removed two rows with missing vals

0.5620989787792113


To fill in this missing data, let's first extract  Iowa County and Iowa County Numbers from this pdf taken here: https://tax.iowa.gov/sites/default/files/2020-07/Iowa%20County%20Names%20and%20Numbers.pdf 

In [10]:
from PyPDF2 import PdfReader

def extract_data_from_pdf(pdf_path):
    # Create a PdfReader object
    pdf = PdfReader(pdf_path)

    county_numbers = []
    county_names = []

    # Extract text from the first page of the PDF
    text = pdf.pages[0].extract_text()
    lines = text.split('\n')
    for line in lines:
        if '-' in line and '76-002' not in line:  # Exclude line with '76-002'
            county_number, county_name = line.split('-', 1)
            county_numbers.append(int(county_number.strip()))  # Convert to int64
            county_names.append(county_name.strip().upper())  # Convert to uppercase
        elif 'O’BRIEN' in line:
            parts = line.split('\t')
            county_numbers.append(parts[0].strip())
            county_names.append(parts[1].strip().upper())

    # Create a DataFrame
    df = pd.DataFrame({
        'County Number': county_numbers,
        'County': county_names
    })

    # Delete rows where 'County Number' equals 33, 66, and 71
    df = df[~df['County Number'].isin([33, 66, 71])]

    # Manually set new entries for 33, 66, and 71
    new_entries = [
        {'County Number': 33, 'County': 'FAYETTE'},
        {'County Number': 34, 'County': 'FLOYD'},
        {'County Number': 66, 'County': 'MITCHELL'},
        {'County Number': 67, 'County': 'MONONA'},
        {'County Number': 71, 'County': "O'BRIEN"}
    ]

    # Create a DataFrame from the new entries
    new_entries_df = pd.DataFrame(new_entries)

    # Concatenate the original DataFrame and the new entries DataFrame
    df = pd.concat([df, new_entries_df], ignore_index=True)

    # Sort the DataFrame based on 'County Number' column in ascending order
    df = df.sort_values('County Number', ascending=True)

    return df


In [11]:
#file path
path = 'Iowa County Names and Numbers.pdf'
#instantiate df countaining county data
county_df = extract_data_from_pdf(path)
#output data to csv if desired
#county_df.to_csv('counties.csv', sep= '\t')

Now we will iterate through the dataframes containing missing county data (those with either county or county number missing, not both), and use the paired county information in each row to fill in the relevant missing data. We start by creating two dictionaries for county number and county. Then, we iterate through each dataframe and check whether the year is 2022 or 2023, since these dfs contain the relevant rows with missing county data. Then, we check if either the missing value is county or county number and use that to fill in the missing data. 

In [12]:
#creating dictionarie for quick lookup of relevant county or county number data.
county_number_to_county = county_df.set_index('County Number')['County'].to_dict()
county_to_county_number = county_df.set_index('County')['County Number'].to_dict()

#get list of dataframes
dataframes_names = list(dataframes_by_year)

#iterate through every year in yearly dataframes
for dataframe_name in dataframes_names:
    #get year of dataframe
    year = int(dataframe_name[-4:])  #

    #check whether year is 2022 or 2023 to speed up process
    if year in [2022, 2023]:
        df = dataframes_by_year[dataframe_name]
        for index, row in df.iterrows():
            #check whether row meets XOR criteria
            if pd.isnull(row['County']) ^ pd.isnull(row['County Number']):
                #if county is missing, fill it using county number from the dictionary
                if pd.isnull(row['County']):
                    county_number = row['County Number']
                    if county_number in county_number_to_county:
                        df.at[index, 'County'] = county_number_to_county[county_number]
                #if county number is missing, fill it using county from the dictionary
                else:
                    county_name = row['County']
                    if county_name in county_to_county_number:
                        df.at[index, 'County Number'] = county_to_county_number[county_name]

        print(f"Processing DataFrame '{dataframe_name}' completed for year {year}.")
    else:
        print(f"Skipping DataFrame '{dataframe_name}' for year {year}.")

Skipping DataFrame 'Iowa_Liquor_Data_2012' for year 2012.
Skipping DataFrame 'Iowa_Liquor_Data_2013' for year 2013.
Skipping DataFrame 'Iowa_Liquor_Data_2014' for year 2014.
Skipping DataFrame 'Iowa_Liquor_Data_2015' for year 2015.
Skipping DataFrame 'Iowa_Liquor_Data_2016' for year 2016.
Skipping DataFrame 'Iowa_Liquor_Data_2017' for year 2017.
Skipping DataFrame 'Iowa_Liquor_Data_2018' for year 2018.
Skipping DataFrame 'Iowa_Liquor_Data_2019' for year 2019.
Skipping DataFrame 'Iowa_Liquor_Data_2020' for year 2020.
Skipping DataFrame 'Iowa_Liquor_Data_2021' for year 2021.
Processing DataFrame 'Iowa_Liquor_Data_2022' completed for year 2022.
Processing DataFrame 'Iowa_Liquor_Data_2023' completed for year 2023.


Let's rerun this earlier block of code to double check that we successfully filled in missing county data

In [13]:
#dictionary to store the missing county data counts for each year
counts_by_year = {}

#iterate through yearly dataframes
for year, df in dataframes_by_year.items():
    #get num rows satisfying missing county condition
    count_condition = (df['County'].isnull() ^ df['County Number'].isnull())
    count = count_condition.sum()  # Count the number of True values

    #store count
    counts_by_year[year] = count

    #output ccount
    print(f"Count of missing county data from {year} : {count}")

Count of missing county data from Iowa_Liquor_Data_2012 : 0
Count of missing county data from Iowa_Liquor_Data_2013 : 0
Count of missing county data from Iowa_Liquor_Data_2014 : 0
Count of missing county data from Iowa_Liquor_Data_2015 : 0
Count of missing county data from Iowa_Liquor_Data_2016 : 0
Count of missing county data from Iowa_Liquor_Data_2017 : 0
Count of missing county data from Iowa_Liquor_Data_2018 : 0
Count of missing county data from Iowa_Liquor_Data_2019 : 0
Count of missing county data from Iowa_Liquor_Data_2020 : 0
Count of missing county data from Iowa_Liquor_Data_2021 : 0
Count of missing county data from Iowa_Liquor_Data_2022 : 0
Count of missing county data from Iowa_Liquor_Data_2023 : 0


Next, it came to my attention that a number of city names in the original dataset were misspelled. Let's identify them. We will do this by downloading the Iowa Populated Places dataset from here: https://geodata.iowa.gov/datasets/iowa-populated-places/explore 

We will also need to download the Iowa Liquor Stores Dataset for reasons that will become clear later on. This can be found here: https://data.iowa.gov/Regulation/Iowa-Liquor-Stores/ykb6-ywnd 

Column names in the liquor_stores dataframe are adjusted for easier use later on. 

In [14]:
iowa_pop_places = pd.read_csv('Iowa_Populated_Places.csv')
iowa_pop_places.head()

,X,Y,FID,PLACES_ID,PLACE_NAME,NAME_CAPS,OTHER_NAME,CO_NAME,CO_SEAT,QUAD_NAME,TYPE,CO_NUMBER,Longitude,Latitude,typedescription
0,-96.556444,43.468872,1,1,Granite,GRANITE,,LYON,,KLONDIKE,3,60,-96.556444,43.468872,Unincorporated town or place name
1,-96.100309,43.476366,2,2,Midland,MIDLAND,,LYON,,EDNA,10,60,-96.100309,43.476366,GNIS place name not referenced by USGS or IDOT...
2,-91.289585,43.496649,3,3,New Albin,NEW ALBIN,,ALLAMAKEE,,NEW ALBIN,1,3,-91.289585,43.496649,Incorporated city or town
3,-94.791935,43.491636,4,4,Huntington,HUNTINGTON,,EMMET,,ESTHERVILLE,3,32,-94.791935,43.491636,Unincorporated town or place name
4,-96.434216,43.453596,5,5,Larchwood,LARCHWOOD,,LYON,,LARCHWOOD,1,60,-96.434216,43.453596,Incorporated city or town


In [15]:
liquor_stores = pd.read_csv('Iowa_Liquor_Stores.csv')
liquor_stores = liquor_stores.drop(['Store Status', 'Report Date'], axis=1)
liquor_stores.rename(columns={'Store Address': 'Store Location'}, inplace=True)
liquor_stores.rename(columns={'Store': 'Store Number'}, inplace=True)
liquor_stores.rename(columns={'Name': 'Store Name'}, inplace=True)
liquor_stores.head()

,Store Number,Store Name,Address,City,State,Zip Code,Store Location
0,10261,AVENUE G STORE / COUNCIL BLUFFS,1602 AVENUE G,COUNCIL BLUFFS,IA,51501,POINT (-95.868007008 41.268345002)
1,4191,FAREWAY STORES #995 / PELLA,2010 WASHINGTON ST,PELLA,IA,50219,POINT (-92.941820991 41.413265006)
2,5268,THE DEPOT ATKINS,"188, PARKRIDGE RD",ATKINS,IA,52206,POINT (-91.862706026 41.992304983)
3,5929,D&T'S / ODEBOLT,417 1ST ST,ODEBOLT,IA,51458,POINT (-95.255976028 42.313622014)
4,6150,THE DEPOT GILMAN,102 S. ELM ST.,GILMAN,IA,50106,POINT (-92.785956987 41.879548017)


Let's start by identifying which rows in our liquor_data don't have a corresponding match in the Iowa Populated Places Dataset (i.e. they have been mispelled). We will start by selecting only the city columns of each. In the liquor data, the city column is called 'CITY' and in the 'Iowa Populated Places' Dataset, it is called 'NAME_CAPS'. We will right join on 'CITY to identify rows that have an error. 

In [16]:
#get city columns only in liquor data and iowa populated places data
liquor_city_data = liquor_data[['City']].copy()
iowa_city_data = iowa_pop_places[['NAME_CAPS']].copy()

#merge them together
merged = iowa_city_data.merge(liquor_city_data, how='right', left_on='NAME_CAPS', right_on='City')

#filter rows which don't have a corresponding match
rows_without_matches = merged[merged['NAME_CAPS'].isnull()]

#how many rows contain mismatches?
len(rows_without_matches)

506500

Let's output which City values that are incorrectly spelled. 

In [17]:
rows_without_matches['City'].value_counts()

City
ARNOLDS PARK        86092
DEWITT              67796
LEMARS              44794
OTTUWMA             40246
CLEARLAKE           33857
MT VERNON           28649
MT PLEASANT         22720
LAKE VIEW           21598
LECLAIRE            19058
ST ANSGAR           15684
MELCHER-DALLAS      14173
ARNOLD'S PARK        8171
ST LUCAS             6868
ST CHARLES           3546
GRAND MOUNDS         2895
GUTTENBURG           2593
KELLOG               2112
OTUMWA               1516
LONETREE              851
FT. ATKINSON          566
COLORADO SPRINGS        2
Name: count, dtype: int64

A couple of things jump out. First of all, there are actually a couple notable errors in the Iowa Populated places dataset. It incorrectly labels ARNOLDS PARK as 'ARNOLD PARK', however, it did highlight that the need to correct 'ARNOLD'S PARK to ARNOLDS PARK. 

DEWITT is correctly spelled as one word in our original dataset, but not in 'Iowa Populated Places'. 

LAKE VIEW is also correctly spelled as two words in our liquor dataset, while it is incorrect in the 'Iowa Populated Places' dataset. This begs the question on whether DE WITT and LAKEVIEW also appear in our liquor dataset.  

Next, LEMARS, CLEARLAKE, LECLAIRE, LONETREE all need spaces. GUTTENBURG, OTUMWA, OTTUWMA, GRAND MOUNDS, and KELLOG are misspelled and COLORADO SPRINGS is not a city in Iowa, but we already addressed this.

Finally we will have to deal with inconsistencies in our abbreviated terms. 

First, let's check how potentially abbreviated terms appear in our liquor dataset. We will also check whether the incorrectly spelled 'LAKEVIEW' and 'DE WITT' cities appear in our original liquor dataset.

In [18]:
#values to match in the city column
specific_cities = ['VERNON', 'PLEASANT', 'ANSGAR', 'LUCAS', 'FORT', 'FT.', 'SAINT', 'LAKEVIEW', 'DE WITT']

#create a mask for rows containing specified values
mask = liquor_city_data['City'].str.contains('|'.join(specific_cities), case=False, na=False)

#store values city values
specific_cities_df = liquor_city_data[mask].copy()

specific_cities_df['City'].value_counts()

City
FORT DODGE         311482
FORT MADISON       119470
PLEASANT HILL      109047
MOUNT VERNON        96708
MOUNT PLEASANT      79859
MT VERNON           28649
MT PLEASANT         22720
ST ANSGAR           15684
FORT ATKINSON       13556
PLEASANTVILLE        9095
ST LUCAS             6868
AFTON                3531
PLEASANT VALLEY      1451
SAINT ANSGAR         1393
FT. ATKINSON          566
Name: count, dtype: int64

As we can see, there are inconsistencies in how the same town are spelled in the data. We will need to correct this. To avoid any confusion, we convert 'MT' to 'MOUNT, 'ST' to SAINT', 'FT.' to 'FORT', etc. 'DE WITT' and 'LAKE VIEW' do not appear in our original dataset, which is nice. 

To summarize here are all of the differnt naming changes that need to happen:

ARNOLD'S PARK --> ARNOLDS PARK \
ARNOLD PARK --> ARNOLDS PARK \
OTTUWMA --> OTTUMWA \
OTUMWA --> OTTUMWA \
GUTTENBURG --> GUTTENBERG \
KELLOG --> KELLOGG \
GRAND MOUNDS --> GRAND MOUND \
LEMARS --> LE MARS \
CLEARLAKE --> CLEAR LAKE \
LECLAIRE --> LE CLAIRE \
LONETREE --> LONE TREE \
FT. ATKINSON --> FORT ATKINSON \
MT VERNON --> MOUNT VERNON \
MT PLEASANT --> MOUNT PLEASANT \
ST ANGSAR --> SAINT ANSGAR 

Now we need to check whether the liquor stores dataset contains mismatches and correct them.

In [19]:
#liquor store city data
liquor_store_city_data = liquor_stores[['City']].copy()

#merge them together
merged_liquor_stores = iowa_city_data.merge(liquor_store_city_data, how='right', left_on='NAME_CAPS', right_on='City')

#filter rows which don't have a corresponding match
liquor_store_cities_without_matches = merged_liquor_stores[merged_liquor_stores['NAME_CAPS'].isnull()]

In [20]:
liquor_store_cities_without_matches['City'].value_counts()

City
DEWITT            7
CLEARLAKE         4
ARNOLDS PARK      3
LECLAIRE          3
LAKE VIEW         3
ST ANSGAR         2
MELCHER-DALLAS    2
MT PLEASANT       2
BEND              1
ST LUCAS          1
ARNOLD'S PARK     1
ST CHARLES        1
GRAND MOUNDS      1
Name: count, dtype: int64

As we can see, there are similar inconsistencies in this dataset as well. Let's create a mapping to correct these issues.

In [21]:
#mapping to correct misspelling.
city_replacements = {
        "ARNOLD'S PARK": 'ARNOLDS PARK',
        'ARNOLD PARK': 'ARNOLDS PARK',
        'OTTUWMA': 'OTTUMWA',
        'OTUMWA': 'OTTUMWA',
        'GUTTENBURG': 'GUTTENBERG',
        'KELLOG': 'KELLOGG',
        'GRAND MOUNDS': 'GRAND MOUND',
        'LEMARS': 'LE MARS',
        'CLEARLAKE': 'CLEAR LAKE',
        'LECLAIRE': 'LE CLAIRE',
        'LONETREE': 'LONE TREE',
        'FT. ATKINSON': 'FORT ATKINSON',
        'MT VERNON': 'MOUNT VERNON',
        'MT PLEASANT': 'MOUNT PLEASANT',
        'ST ANSGAR': 'SAINT ANSGAR',
        'ST LUCAS': 'SAINT LUCAS',
        'ST CHARLES': 'SAINT CHARLES'
        
    }

In [22]:
#loop through dataframes in df list
for name, df in dataframes_by_year.items():
    print(f"Processing dataFrame: {name}")
    
    #loop to correct misspelling
    for incorrect_city, correct_city in city_replacements.items():
        df['City'] = df['City'].replace(incorrect_city, correct_city)
    
    #update yearly dataframe
    dataframes_by_year[name] = df

Processing dataFrame: Iowa_Liquor_Data_2012
Processing dataFrame: Iowa_Liquor_Data_2013
Processing dataFrame: Iowa_Liquor_Data_2014
Processing dataFrame: Iowa_Liquor_Data_2015
Processing dataFrame: Iowa_Liquor_Data_2016
Processing dataFrame: Iowa_Liquor_Data_2017
Processing dataFrame: Iowa_Liquor_Data_2018
Processing dataFrame: Iowa_Liquor_Data_2019
Processing dataFrame: Iowa_Liquor_Data_2020
Processing dataFrame: Iowa_Liquor_Data_2021
Processing dataFrame: Iowa_Liquor_Data_2022
Processing dataFrame: Iowa_Liquor_Data_2023


In [23]:
#update liquor store data
for incorrect_city, correct_city in city_replacements.items():
    liquor_stores['City'] = liquor_stores['City'].replace(incorrect_city, correct_city)

Now let's inspect one of the yearly dataframes. Here we can see that Zip Code and Item Number are of type object, indicating that these might have mismatched datatypes, given the prior warnings. We will iterate through the Zip Code and Item Number colums to detect non-numeric characters.

In [24]:
liquor_2012 = dataframes_by_year['Iowa_Liquor_Data_2012']
liquor_2012.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2082059 entries, 3815854 to 26568553
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Store Number           int64  
 2   Store Name             object 
 3   Address                object 
 4   City                   object 
 5   Zip Code               object 
 6   Store Location         object 
 7   County Number          float64
 8   County                 object 
 9   Category               float64
 10  Category Name          object 
 11  Vendor Number          float64
 12  Vendor Name            object 
 13  Item Number            object 
 14  Item Description       object 
 15  Pack                   int64  
 16  Bottle Volume (ml)     int64  
 17  State Bottle Cost      float64
 18  State Bottle Retail    float64
 19  Bottles Sold           int64  
 20  Sale (Dollars)         float64
 21  Volume Sold (Liters)   float64
 22  Volume Sold (Gal

In [25]:
#regex pattern to find rows with non-numeric values(excudling '.')
pattern = r'[^0-9\.]'  

#list to store dfs with non_numeric values
dataframes_with_non_numeric_values = []

#loop to iterate through yearly dataframes over 'Zip Code'
for df_name, df in dataframes_by_year.items():

    #mask to first filter out na values in zip code (we will deal with those later)
    non_na_mask_zip = df['Zip Code'].notna()

    #get rows containing non-numeric values
    non_numeric_rows_zip = df[non_na_mask_zip & df['Zip Code'].astype(str).str.contains(pattern, regex=True, na=False)]

    #store non-numeric rows in a list
    if not non_numeric_rows_zip.empty:
        dataframes_with_non_numeric_values.append(non_numeric_rows_zip)

#concatentate list containing non-numeric rows together in a dataframe. 
if dataframes_with_non_numeric_values:
    concatenated_dunlap_df = pd.concat(dataframes_with_non_numeric_values)
    print("Non-numeric values detected in 'Zip Code' column in yearly dataframes (excluding NA values):")
else:
    print("No rows with non-numeric values detected in 'Zip Code' column (excluding NA values).")


Non-numeric values detected in 'Zip Code' column in yearly dataframes (excluding NA values):


Checking the value counts for 'Zip Code' and 'City', we see that there was an error in entering the data for the city of 'DUNLAP'. 

In [26]:
concatenated_dunlap_df['Zip Code'].value_counts()

Zip Code
712-2    7940
Name: count, dtype: int64

In [27]:
concatenated_dunlap_df['City'].value_counts()

City
DUNLAP    7940
Name: count, dtype: int64

Now we need to loop over the 'Item Number' column to detect mismatched datatypes:

In [28]:
#regex pattern to find rows with non-numeric values(excudling '.')
pattern = r'[^0-9\.]' 

#list to store dfs with non_numeric values
dataframes_with_non_numeric_values_item = []

#loop to iterate through yearly dataframes over 'Item Number'
for df_name, df in dataframes_by_year.items():
     #mask to first filter out na values in item number
    non_na_mask_item = df['Item Number'].notna()

    #get rows containing non-numeric values
    non_numeric_rows_item = df[non_na_mask_item & df['Item Number'].astype(str).str.contains(pattern, regex=True, na=False)]

    #store non-numeric rows in a list
    if not non_numeric_rows_item.empty:
        dataframes_with_non_numeric_values_item.append(non_numeric_rows_item)

#concatentate list containing non-numeric rows together in a dataframe. 
if dataframes_with_non_numeric_values_item:
    concatenated_df_bad_item = pd.concat(dataframes_with_non_numeric_values_item)
    print("Non-numeric values detected in 'Item Number' column in yearly dataframes (excluding NA values):")
else:
    print("No rows with non-numeric values detected in 'Item Number' column (excluding NA values).")


Non-numeric values detected in 'Item Number' column in yearly dataframes (excluding NA values):


In [29]:
concatenated_df_bad_item.head()

,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
8293604,10/11/2016,2619,HY-VEE WINE AND SPIRITS / WDM,1725 74TH ST,WEST DES MOINES,50266,POINT (-93.808855 41.598515),77.0,POLK,1901200.0,...,x904631,TANQUERAY GIN MINI - USE 904631 CODE,12,500,14.55,21.83,12,261.96,6.0,1.59
9299557,04/06/2017,4988,HAPPY'S WINE & SPIRITS,5925 UNIVERSITY AVE,CEDAR FALLS,50613.0,POINT (-92.429331 42.512766),7.0,BLACK HAWK,1901200.0,...,x904631,TANQUERAY GIN MINI - USE 904631 CODE,12,500,14.55,21.83,12,261.96,6.0,1.59
9319467,04/04/2017,2665,HY-VEE / WAUKEE,1005 E HICKMAN RD,WAUKEE,50263.0,POINT (-93.854477 41.615059),25.0,DALLAS,1901200.0,...,x904631,TANQUERAY GIN MINI - USE 904631 CODE,12,500,14.55,21.83,12,261.96,6.0,1.59


As we can see below, there are exactly 3 data points which have 'x904631' instead of a numeric value.

Now we will loop through and correct these issues. We will simply replace any values in the Zip Code column containing '712-2' with 51529 since they only occur in rows where the City is DUNLAP. Then we will the remove the tiny number of rows containing non-numeric data types in the 'Item Number' column for simplicity. 

In [30]:

#counters to track removed rows for bad item numbers and number of zip code values changed
bad_inum_removed_rows_counter = 0
changed_zip_code_counter = 0

#loop through dataframes in year dataframes list
for name, df in dataframes_by_year.items():
    print(f"Processing DataFrame: {name}")
    print(f"Length before processing: {len(df)}")
    
    #count rows before the conversion
    rows_before_conversion = df[df['Zip Code'] == '712-2'].shape[0]
    print(f"rows before zip code conversion: {rows_before_conversion}")


    #change 'Zip Code' value '712-2' to 51529
    df.loc[df['Zip Code'] == '712-2', 'Zip Code'] = 51529
    rows_after_conversion = df[df['Zip Code'] == '712-2'].shape[0]
    print(f"rows containing bad zip after zip code conversion: {rows_after_conversion}")

    total_converted = rows_before_conversion - rows_after_conversion
    changed_zip_code_counter += total_converted

    #remove rows where 'Item Number' == 'x904631' and update counter
    removed_rows = df[df['Item Number'] == 'x904631'].index
    bad_inum_removed_rows_counter += len(removed_rows)
    df.drop(removed_rows, inplace=True)
    
    #update yearly dataframe in list
    dataframes_by_year[name] = df
    
    print(f"Length after processing: {len(df)}")

#print total number of removed rows and changed zip codes
print(f"\nTotal removed rows where 'Item Number' == 'x904631': {bad_inum_removed_rows_counter}")
print(f"Total changed 'Zip Code' values: {changed_zip_code_counter}")

Processing DataFrame: Iowa_Liquor_Data_2012
Length before processing: 2082059
rows before zip code conversion: 1485
rows containing bad zip after zip code conversion: 0
Length after processing: 2082059
Processing DataFrame: Iowa_Liquor_Data_2013
Length before processing: 2063763
rows before zip code conversion: 1614
rows containing bad zip after zip code conversion: 0
Length after processing: 2063763
Processing DataFrame: Iowa_Liquor_Data_2014
Length before processing: 2097796
rows before zip code conversion: 1780
rows containing bad zip after zip code conversion: 0
Length after processing: 2097796
Processing DataFrame: Iowa_Liquor_Data_2015
Length before processing: 2184483
rows before zip code conversion: 1725
rows containing bad zip after zip code conversion: 0
Length after processing: 2184483
Processing DataFrame: Iowa_Liquor_Data_2016
Length before processing: 2279893
rows before zip code conversion: 1336
rows containing bad zip after zip code conversion: 0
Length after processing

Nw I am ready to separate rows containing NaN values from the other yearly dataframes into a single dataframe. I will call this dataframe cumulative_errors.

In [31]:
#create empty dataframe to store rows containing NA values
cumulative_errors = pd.DataFrame()

#iterate through dataframes in yearly dataframes list
for name, df in dataframes_by_year.items():
    print(f"Processing {name}...")
    
    #print length before removing Na rows
    print(f"Length before removing NaN rows: {len(df)}")
    
    #identify rows with NA values
    nan_rows = df[df.isna().any(axis=1)]
    
    #concatenate nan_rows to cumulative errors
    cumulative_errors = pd.concat([cumulative_errors, nan_rows], ignore_index=True)
    
    #remove nan rows from original yearly dataframe
    df.dropna(inplace=True)
    
    #print the length after removing Na rows to ensure we are actually separating that data out.
    print(f"Length after removing NaN rows: {len(df)}")

#print number of rows containing na values
print(f"\nLength of cumulative_errors: {len(cumulative_errors)}")

Processing Iowa_Liquor_Data_2012...
Length before removing NaN rows: 2082059
Length after removing NaN rows: 1884232
Processing Iowa_Liquor_Data_2013...
Length before removing NaN rows: 2063763
Length after removing NaN rows: 1872144
Processing Iowa_Liquor_Data_2014...
Length before removing NaN rows: 2097796
Length after removing NaN rows: 1909019
Processing Iowa_Liquor_Data_2015...
Length before removing NaN rows: 2184483
Length after removing NaN rows: 1984108
Processing Iowa_Liquor_Data_2016...
Length before removing NaN rows: 2279892
Length after removing NaN rows: 1962631
Processing Iowa_Liquor_Data_2017...
Length before removing NaN rows: 2291274
Length after removing NaN rows: 2028321
Processing Iowa_Liquor_Data_2018...
Length before removing NaN rows: 2355556
Length after removing NaN rows: 2129085
Processing Iowa_Liquor_Data_2019...
Length before removing NaN rows: 2380345
Length after removing NaN rows: 2158261
Processing Iowa_Liquor_Data_2020...
Length before removing NaN r

Before going any further, let's try to convert the Zip Code and Item Number column types to ensure no lingering issues persist. After doing the conversion, we will drop rows with na values. If we don't drop any rows, then the conversion was a success. 

In [32]:
#iterate through yearly dataframes
for name, df in dataframes_by_year.items():
    print(f"Processing DataFrame: {name}")

    initial_length = len(df)
    print(f"Length before processing: {len(df)}")

    #convert 'Zip Code' to type int 
    df['Zip Code'] = pd.to_numeric(df['Zip Code'], errors='coerce').astype('Int64')

    #convert 'Item Number' to type int
    df['Item Number'] = pd.to_numeric(df['Item Number'], errors='coerce').astype('Int64')

    #drop rows with Na values
    df.dropna(inplace=True)
    final_length = len(df)

    len_bool = (final_length - initial_length == 0)

    print(f'No rows lost for {name}: {len_bool}')

    print(f"Length after processing: {len(df)}\n")

Processing DataFrame: Iowa_Liquor_Data_2012
Length before processing: 1884232
No rows lost for Iowa_Liquor_Data_2012: True
Length after processing: 1884232

Processing DataFrame: Iowa_Liquor_Data_2013
Length before processing: 1872144
No rows lost for Iowa_Liquor_Data_2013: True
Length after processing: 1872144

Processing DataFrame: Iowa_Liquor_Data_2014
Length before processing: 1909019
No rows lost for Iowa_Liquor_Data_2014: True
Length after processing: 1909019

Processing DataFrame: Iowa_Liquor_Data_2015
Length before processing: 1984108
No rows lost for Iowa_Liquor_Data_2015: True
Length after processing: 1984108

Processing DataFrame: Iowa_Liquor_Data_2016
Length before processing: 1962631
No rows lost for Iowa_Liquor_Data_2016: True
Length after processing: 1962631

Processing DataFrame: Iowa_Liquor_Data_2017
Length before processing: 2028321
No rows lost for Iowa_Liquor_Data_2017: True
Length after processing: 2028321

Processing DataFrame: Iowa_Liquor_Data_2018
Length before 

With the rows containing missing values separated, let's investigate this dataset further.

In [33]:
len(cumulative_errors)

2590496

In [34]:
cumulative_errors.head()

,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,08/06/2012,2643,HY-VEE WINE AND SPIRITS / WATERLOO,2126 KIMBALL AVE,WATERLOO,50701.0,NaN,7.0,BLACK HAWK,1081700.0,...,76040,MIDNIGHT MOON STRAWBERRY,6,750,10.42,15.63,6,93.78,4.50,1.19
1,04/25/2012,2535,HY-VEE FOOD STORE #1 / WDM,1700 VALLEY WEST DR,WEST DES MOINES,50265.0,NaN,77.0,POLK,1032080.0,...,34433,GREY GOOSE VODKA,12,750,17.97,26.96,5,134.80,3.75,0.99
2,07/19/2012,3912,SMOKIN' JOE'S #14 TOBACCO AND LIQUOR,225 EDGEWOOD RD,CEDAR RAPIDS,52405.0,NaN,57.0,LINN,1011200.0,...,19063,JIM BEAM,48,200,2.85,4.28,6,25.68,1.20,0.32
3,11/05/2012,4410,KUM & GO #203 / PERRY,1219 1ST ST,PERRY,50220.0,NaN,8.0,BOONE,1062310.0,...,43331,CAPTAIN MORGAN SPICED RUM MINI,12,500,5.50,8.25,2,16.50,1.00,0.26
4,04/17/2012,4262,CORK AND BOTTLE / CARROLL,1004 HIGHWAY 30 W,CARROLL,51401.0,NaN,14.0,CARROLL,1032080.0,...,34578,PINNACLE VODKA,6,1750,10.43,16.14,1,16.14,1.75,0.46


To help orient our data cleaning strategy, let's see which columns contain NA values.

In [35]:
print(cumulative_errors.columns[cumulative_errors.isna().any()])

Index(['Address', 'City', 'Zip Code', 'Store Location', 'County Number',
       'County', 'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'State Bottle Cost', 'State Bottle Retail', 'Sale (Dollars)'],
      dtype='object')


Let's start by addressing rows which contain missing data in either the State Bottle Cost, State Bottle Retail, Vendor Number, and Vendor Name columns. There are a grand total of 19 rows out of over 27 million data points that fit this criteria. For the sake of simplicity, I will remove them for now. However, all data points removed going forward (including this case), will be stored in a separate dataframe called the_pile for further refinement.

In [36]:
the_pile = cumulative_errors[cumulative_errors[['State Bottle Cost', 'State Bottle Retail', 'Vendor Number', 'Vendor Name']].isnull().any(axis=1)]
len(the_pile)

19

In [37]:
#Adding in earlier removed rows based on bad 'Item Number' data
the_pile = pd.concat([the_pile, concatenated_df_bad_item])

In [38]:
#This removes an entire 19 rows in the whole dataset and makes life much easier
cumulative_errors = cumulative_errors.dropna(subset=['State Bottle Cost', 'State Bottle Retail'], how='any')
cumulative_errors = cumulative_errors.dropna(subset=['Vendor Number', 'Vendor Name'], how='any')
len(cumulative_errors)

2590477

Let's examine which rows still contain NA values after removal of those rows:

In [39]:
print(cumulative_errors.columns[cumulative_errors.isna().any()])

Index(['Address', 'City', 'Zip Code', 'Store Location', 'County Number',
       'County', 'Category', 'Category Name'],
      dtype='object')


Next let's address the Category, and Category Name columns. If we subset the rows which contain either a 'Category' or 'Category Name' column, we find that there are 25040 rows. This isn't ideal, but in the interest of time, I am simply going to move those rows to the_pile. The difficulty here is that 'Category' and 'Category Name' aren't always stable for the same 'Item Description', making it somewhat difficult to determine the best way of handling these missing values. Still, this data represents less than 0.1 percent of total data. 

In [40]:
#getting value counts of 'Item Description' for rows where 'Category' or 'Category Name' are null.
category_nan_df = cumulative_errors[cumulative_errors['Category'].isnull() | cumulative_errors['Category Name'].isnull()]
print(len(category_nan_df))

25040


In [41]:
#Adding in earlier removed rows based on bad 'Item Number' data
the_pile = pd.concat([the_pile, category_nan_df])
len(the_pile)

25062

In [42]:
#drop na rows in 'Category; and 'Category Name' columns for now
cumulative_errors = cumulative_errors.dropna(subset=['Category', 'Category Name'], how='any')
print(cumulative_errors.columns[cumulative_errors.isna().any()])

Index(['Address', 'City', 'Zip Code', 'Store Location', 'County Number',
       'County'],
      dtype='object')


Let's move on to the County and County Number Columns. Any rows that contain a missing County value will also contain a missing County Number column and vice versa.

In [43]:
county_nan_df = cumulative_errors[cumulative_errors['County'].isnull() & cumulative_errors['County Number'].isnull()]

In [44]:
len(county_nan_df)

159120

For simplicity sake, let's examine the cities within this subset. If these cities span only a single county, then we can performa simple mapping to fill in those missing values. If they do not, we cannot safely map county and county number to those rows.

In [45]:
total_county_missing = county_nan_df['City'].value_counts()

In [46]:
total_county_missing.head(30)

City
DES MOINES         7575
BELMOND            6072
DAVENPORT          4672
SIOUX CITY         3732
GUTTENBERG         3340
CEDAR RAPIDS       3198
ANAMOSA            2729
CRESCO             2423
IOWA CITY          2295
MASON CITY         2221
COUNCIL BLUFFS     2208
HAMPTON            2153
HARLAN             2018
DUBUQUE            1878
CLARINDA           1866
FORT DODGE         1779
OTTUMWA            1609
CORALVILLE         1484
INDIANOLA          1399
MARSHALLTOWN       1158
WAVERLY            1112
MUSCATINE          1038
WATERLOO           1028
BOONE               952
ROCKWELL            920
EVANSDALE           916
MISSOURI VALLEY     904
CLINTON             885
GLENWOOD            879
DUNLAP              865
Name: count, dtype: int64

Des Moines, Cedar Rapids, Sioux City, and Davenport span multiple counties, so we unfortuantely cannot risk mapping county numbers to them. However, we can safely perform the following mapping as these cities lie entirely within their respective counties:

In [47]:
#Critera for manual mapping: population < 5000 & missing count > 1500

mapping_values = {
    'BELMOND': {'County': 'WRIGHT', 'County Number': 99.0},
    'COUNCIL BLUFFS': {'County': 'POTTAWATTAMIE', 'County Number': 78.0},
    'GUTTENBERG': {'County': 'CLAYTON', 'County Number': 22.0},
    'ANAMOSA': {'County': 'JONES', 'County Number': 53.0},
    'CRESCO': {'County': 'HOWARD', 'County Number': 45.0},
    'HAMPTON': {'County': 'FRANKLIN', 'County Number': 35.0},
    'HARLAN': {'County': 'SHELBY', 'County Number': 83.0},
    'CLARINDA': {'County': 'PAGE', 'County Number': 73.0},
    'FORT DODGE': {'County': 'WEBSTER', 'County Number': 94.0},
    'CEDAR RAPIDS': {'County': 'LINN', 'County Number': 57.0},
    'MASON CITY': {'County': 'CERRO GORDO', 'County Number': 17.0},
    'IOWA CITY': {'County': 'JOHNSON', 'County Number': 52.0},
    'DUBUQUE': {'County': 'DUBUQUE', 'County Number': 31.0},
    'MARSHALLTOWN': {'County': 'MARSHALL', 'County Number': 64.0},
    'WAVERLY': {'County': 'BREMER', 'County Number': 9.0},
    'MUSCATINE': {'County': 'MUSCATINE', 'County Number': 70.0},
    'WATERLOO': {'County': 'BLACK HAWK', 'County Number': 7.0},
    'BOONE': {'County': 'BOONE', 'County Number': 15.0},
    'COLUMBUS JUNCTION': {'County': 'LOUISA', 'County Number': 58.0},
    'CORALVILLE': {'County': 'JOHNSON', 'County Number': 52.0},
    'CLINTON': {'County': 'CLINTON', 'County Number': 23.0},
    'INDIANOLA': {'County': 'WARREN', 'County Number': 91.0}
}


In [48]:
# Update values in other_rows_df based on the mapping
for city, values in mapping_values.items():
    cumulative_errors.loc[cumulative_errors['City'] == city, 'County'] = values['County']
    cumulative_errors.loc[cumulative_errors['City'] == city, 'County Number'] = values['County Number']

# Select rows in county_nan_df where 'County' and 'County Number' are both NaN
county_nan_df = cumulative_errors[cumulative_errors['County'].isnull() & cumulative_errors['County Number'].isnull()]
len(county_nan_df)


115071

After filling in some of those missing values, we still have a sizeable chunk that contain missing county information. However, in the interest of time, shunt the missing data to the pile for now.

In [49]:
#adding bad county data to the pile
the_pile = pd.concat([the_pile, county_nan_df])
len(the_pile)

140133

In [50]:
#removing rows that contain missing county and county number data for now
cumulative_errors = cumulative_errors.dropna(subset=['County', 'County Number'], how='any')

In [ ]:
print(cumulative_errors.columns[cumulative_errors.isna().any()])

Now let's move on to Address and Zip Code. If we subset the remaining data that either contain missing Address or Zip Code fields, there are a 'whopping' 23 rows. Therefore, I feel safe simply moving this data to the_pile for now. 

In [51]:
nan_address_df = cumulative_errors[cumulative_errors['Address'].isnull() | cumulative_errors['Zip Code'].isnull()]

In [52]:
len(nan_address_df)

23

In [53]:
#adding bad address data to the pile
the_pile = pd.concat([the_pile, nan_address_df])
len(the_pile)

140156

In [54]:
#removing rows that contain missing Address or Zip Code data for now
cumulative_errors = cumulative_errors.dropna(subset=['Address', 'Zip Code'], how='any')

In [55]:
print(cumulative_errors.columns[cumulative_errors.isna().any()])

Index(['Store Location'], dtype='object')


Before we delve into this task of filling in geodata, we must address a significant matter. It's essential to ensure that a city linked with the Store Number in the liquor_stores data matches the corresponding city associated with the same store number in the year-specific data. In cases where these cities don't match, relying on the geodata obtained from the liquor store may compromise accuracy. We also need to check for mismatches in the Zip Code as well. 

Quick note: Ensuring there is a complete match between the 'Address' columns for both datasets is way too restrictice. Slight differences in the spelling of the same address could indicate a mismatch. For example, Boston St. and Boston Street are the same address, but it would be incorrect to say we can't use the geodata in the liquor_stores to fill in missing geodata. The code below shows that there are large numbers of store numbers in the liquor_stores data that have at least one mismatched address in the 20+ million rows in the yearly liquor sales data. 

In [56]:
#initialize counters to track whether associated store number addresses are mismatched
matching_address_counter = 0
mismatching_address_counter = 0

#iterate through the store number in liquore stores
for store_num in liquor_stores['Store Number'].unique():
    #instantiate an empty set to collect addresses associated with the store number in both liquor_stores and dataframes_by_year
    all_addresses = set()

    #get the address from liquor_stores for the current store number
    store_addresses = set(liquor_stores.loc[liquor_stores['Store Number'] == store_num, 'Address'])

    #update set with new addresses
    all_addresses.update(store_addresses)

    #iterate through each dataframe in dataframes_by_year
    for _, df in dataframes_by_year.items():
        #first check if store number is in the current dataframe
        if not df[df['Store Number'] == store_num].empty:
            #get all 'addresses associated with the store number in the current dataframe
            year_addresses = set(df.loc[df['Store Number'] == store_num, 'Address'])
            #update set
            all_addresses.update(year_addresses)

    #if address set is equal to 1, the address listed in the liquor_stores data matches all instances in the yearly dataframes
    if len(all_addresses) == 1:
        matching_address_counter += 1
    else:
        mismatching_address_counter += 1

#output counts
print(f"Matching addresses count: {matching_address_counter}")
print(f"Mismatching addresses count: {mismatching_address_counter}")


Matching addresses count: 2286
Mismatching addresses count: 207


However,  it is much more plausible to remove Store Numbers in the liquor_stores data associated with more than 1 zip code or more than 1 city in the yearly liquor sales data. First let's find 'Store Number' in liquor_stores associated with more than 1 'City' in the yearly sales data. 

In [57]:
#initialize counters
matching_city_counter = 0
mismatching_city_counter = 0
#list to store mismatching city store numbers
mismatching_city_store_numbers = []

#iterate through store number column in liquor stores
for store_num in liquor_stores['Store Number'].unique():
    #instantiate an empty set to collect cities associated with the store number in both liquor_stores and dataframes_by_year
    all_cities = set()

    #get city from liquor_stores for the current store number
    store_cities = set(liquor_stores.loc[liquor_stores['Store Number'] == store_num, 'City'])
    #update set with city info in liquor stores
    all_cities.update(store_cities)

    #iterate through yearly dataframes
    for _, df in dataframes_by_year.items():
        #check if the store number exists in the current dataframe
        if not df[df['Store Number'] == store_num].empty:
            #get all cities associated with the store number in the current dataframe
            year_cities = set(df.loc[df['Store Number'] == store_num, 'City'])
            #update set with potentially new cities
            all_cities.update(year_cities)

    #if length of set is 1, then we know the city is the same for all instances in all of the data
    if len(all_cities) == 1:
        matching_city_counter += 1
        
    else:
        #if it is greater than 1, we need to update the counter and add that store num to a list
        mismatching_city_counter += 1
        mismatching_city_store_numbers.append(store_num)

#output results
print(f"Matching cities count: {matching_city_counter}")
print(f"Mismatching cities count: {mismatching_city_counter}")
print(f"Mismatching city Store Numbers: {mismatching_city_store_numbers}")


Matching cities count: 2490
Mismatching cities count: 3
Mismatching city Store Numbers: [5619, 9911, 3822]


Next, we do the exact same thing for Zip Code.

In [58]:
#initialize counters
matching_zip_counter = 0
mismatching_zip_counter = 0
#list to store mismatching city store numbers
mismatching_zip_store_numbers = []

#iterate through store number column in liquor stores
for store_num in liquor_stores['Store Number'].unique():
    #instantiate an empty set to collect zip codes associated with the store number in both liquor_stores and dataframes_by_year
    all_zip_codes = set()

    #get zip code from liquor_stores for the current store number
    store_zip_codes = set(liquor_stores.loc[liquor_stores['Store Number'] == store_num, 'Zip Code'])
    all_zip_codes.update(store_zip_codes)

    #iterate through yearly dataframes
    for _, df in dataframes_by_year.items():
        #check if the store number exists in the current dataframe
        if not df[df['Store Number'] == store_num].empty:
            #get all zip codes associated with the store number in the current dataframe
            year_zip_codes = set(df.loc[df['Store Number'] == store_num, 'Zip Code'])

            #update set with potentially new cities
            all_zip_codes.update(year_zip_codes)

    #if length of set is 1, then we know the zip is the same for all instances in all of the data
    if len(all_zip_codes) == 1:
        matching_zip_counter += 1
        
    else:
        #if it is greater than 1, we need to update the counter and add that store num to a list
        mismatching_zip_counter += 1
        mismatching_zip_store_numbers.append(store_num)

#output results
print(f"Matching zip codes count: {matching_zip_counter}")
print(f"Mismatching zip codes count: {mismatching_zip_counter}")
print(f"Matching zip code Store Numbers: {mismatching_zip_store_numbers}")


Matching zip codes count: 2481
Mismatching zip codes count: 12
Matching zip code Store Numbers: [4722, 5091, 9911, 5097, 3805, 2656, 4457, 4084, 5116, 6084, 2514, 4353]


Because analyzing data at the county level is also important to our analysis, we need to verify that each of the store numbers in liquor_stores is associated with 1 and only 1 county in our yearly data before using it to fill missing data. 

In [59]:
#initialize counters
matching_county_counter = 0
mismatching_county_counter = 0
#list to store mismatching store numbers
mismatching_county_store_numbers = []

#iterate through store number column in liquor stores
for store_num in liquor_stores['Store Number'].unique():
    #instantiate an empty set to collect counties associated with the store number in dataframes_by_year
    all_counties = set()

    #iterate through yearly dataframes
    for _, df in dataframes_by_year.items():
        #check if the store number exists in the current dataframe
        if not df[df['Store Number'] == store_num].empty:
            #get all counties associated with the store number in the current dataframe
            year_counties = set(df.loc[df['Store Number'] == store_num, 'County'])

            #update set with potentially new counties
            all_counties.update(year_counties)

    #if length of set is 1, then we know the county is the same for all instances in all of the data
    if len(all_counties) == 1:
        matching_county_counter += 1
        
    else:
        #if it is greater than 1, we need to update the counter and add that store num to a list
        mismatching_county_counter += 1
        mismatching_county_store_numbers.append(store_num)

#output results
print(f"Matching county count: {matching_county_counter}")
print(f"Mismatching county count: {mismatching_county_counter}")
print(f"Mismatching county Store Numbers: {mismatching_county_store_numbers}")


Matching county count: 2374
Mismatching county count: 119
Mismatching county Store Numbers: [10088, 10278, 9904, 9941, 4067, 5288, 10276, 10225, 3855, 8804, 8802, 9952, 5286, 5336, 5025, 10286, 9949, 10291, 6124, 3461, 10184, 10282, 10266, 5447, 10166, 8807, 9944, 10151, 10108, 10272, 5906, 10277, 10079, 8811, 6166, 6346, 10290, 10156, 10281, 8803, 9953, 10247, 10128, 10285, 6222, 10235, 9948, 5947, 9943, 10092, 10265, 10273, 10228, 8810, 10047, 5201, 5943, 9950, 10289, 2656, 10027, 6213, 10163, 10264, 9050, 6216, 10210, 10274, 9939, 10242, 5104, 10236, 10110, 10280, 8809, 9942, 10246, 5424, 4944, 10182, 10284, 10131, 10169, 10086, 10222, 10031, 6347, 9947, 10173, 8805, 10279, 9940, 10226, 9951, 10123, 10041, 10287, 8801, 9935, 10262, 10275, 10230, 10237, 6281, 6192, 10268, 10271, 5709, 10218, 4320, 8800, 2694, 10087, 9945, 10283, 8806, 10238, 8808, 10119]


Now we will combine the two lists of Store Numbers associated with more than 1 zip code or city. Then we will filter rows with those store numbers out of the original liquor_stores data. 

In [60]:
#merge lists with store numbers that are associated with mismatched cities and zip codes
combined_mismatching_store_numbers = set(mismatching_city_store_numbers + mismatching_zip_store_numbers + mismatching_county_store_numbers)

In [61]:
combined_mismatching_store_numbers

{2514,
 2656,
 2694,
 3461,
 3805,
 3822,
 3855,
 4067,
 4084,
 4320,
 4353,
 4457,
 4722,
 4944,
 5025,
 5091,
 5097,
 5104,
 5116,
 5201,
 5286,
 5288,
 5336,
 5424,
 5447,
 5619,
 5709,
 5906,
 5943,
 5947,
 6084,
 6124,
 6166,
 6192,
 6213,
 6216,
 6222,
 6281,
 6346,
 6347,
 8800,
 8801,
 8802,
 8803,
 8804,
 8805,
 8806,
 8807,
 8808,
 8809,
 8810,
 8811,
 9050,
 9904,
 9911,
 9935,
 9939,
 9940,
 9941,
 9942,
 9943,
 9944,
 9945,
 9947,
 9948,
 9949,
 9950,
 9951,
 9952,
 9953,
 10027,
 10031,
 10041,
 10047,
 10079,
 10086,
 10087,
 10088,
 10092,
 10108,
 10110,
 10119,
 10123,
 10128,
 10131,
 10151,
 10156,
 10163,
 10166,
 10169,
 10173,
 10182,
 10184,
 10210,
 10218,
 10222,
 10225,
 10226,
 10228,
 10230,
 10235,
 10236,
 10237,
 10238,
 10242,
 10246,
 10247,
 10262,
 10264,
 10265,
 10266,
 10268,
 10271,
 10272,
 10273,
 10274,
 10275,
 10276,
 10277,
 10278,
 10279,
 10280,
 10281,
 10282,
 10283,
 10284,
 10285,
 10286,
 10287,
 10289,
 10290,
 10291}

In [62]:
len(liquor_stores)

2493

In [63]:
#filter liquor_stores to remove rows where 'Store Number' appears in the set
filtered_liquor_stores = liquor_stores[~liquor_stores['Store Number'].isin(combined_mismatching_store_numbers)]
len(filtered_liquor_stores)

2361

Before we fill in geodata, let's seperate out the rows containing missing store location info from the rest of the cumulative errors dataframe to prevent us from potentially overwriting data. 

In [64]:
#separating rows with Na only in the store location column
nan_location_df = cumulative_errors[cumulative_errors['Store Location'].isnull() & cumulative_errors.drop('Store Location', axis=1).notnull().all(axis=1)]

#select that do not occur in nan_location_df
non_location_nan_df = cumulative_errors[~cumulative_errors.index.isin(nan_location_df.index)]

In [65]:
len(nan_location_df)

2407088

In [66]:
len(non_location_nan_df)

43255

Now we simply replace the missing geodata with the corresponding geodata in the liquor_stores data. At this point, we've verified that the geodata we are filling in from liquor_stores is associated with 1 and only 1 city, zip code, and county in our yearly data. The only possible error here would occur in the cases where:

- A store has remained in the same city for 12 years AND
- A store has remained within the same zip code for 12 years AND
- A store has remained wthin the same county for 12 years BUT
- At some point, the store has has moved to a different address within the same city, zip code, and county as before.

This event should be exceedingly rare. Less than 10% of the store numbers in liquor_stores are associated with more than 1 address in our yearly data. And of those store numbers, the differences in addresses is not due to an actual difference in geolocation, but rather slight spelling differences. In addition, any actual errors would likely account for a small fraction of each store number associated with the missing data (as the newer address might be present in only a small subset of the data). Moreover, achieving such precise accuracy isn't relevant to this analysis anyway as the goal here is mostly in prototyping. However, if you plan to use the techniques in this notebook to examine liquor consumption at the 'neighborhood level', I would recommend that you either

- Not fill geodata to rows where the Store Number is associated with more than 1 address AND OR
- Validate/harmonize the address data for store numbers.

Even after performing such granular steps, I doubt you would see much change, but I digress. 

In [67]:
#merge nan_location_df with filtered_liquor_stores based on store number
merged_df = nan_location_df.merge(filtered_liquor_stores[['Store Number', 'Store Location']], on='Store Number', how='left')

#update store location in nan_location_df with values from filtered_liquor_stores where matching store number exists
nan_location_df.loc[:, 'Store Location'] = merged_df['Store Location_y'].fillna(nan_location_df['Store Location'])

Although a large chunk of missing data remains after the mapping, I will move this set to the_pile for now. This mapping successfully reduced na values by over 2 million data points.

In [68]:
#calculate the sum of remaining rows containing Na values in store Location column
sum_of_nan_rows = nan_location_df.isna().sum(axis=1).sum()
print(f"Sum of rows containing Na values: {sum_of_nan_rows}")

Sum of rows containing Na values: 366669


In this section, I first remerge the cummulative_errors dataset (now called partially_cleaned_data) after some geodata was filled. Then, I re-identify which rows contain na values and add those to the_pile. The na values are dropped from partially_cleaned_data and now become fully_cleaned_data.

In [69]:
partially_cleaned_data = pd.concat([nan_location_df, non_location_nan_df])

#identify rows with missing values
rows_with_na = partially_cleaned_data[partially_cleaned_data.isnull().any(axis=1)]

#add missing rows to the pile
the_pile = pd.concat([the_pile, rows_with_na])

#remove rows with missing values
fully_clean_data = partially_cleaned_data.dropna()


Now let's get an overview of our 'fully cleaned data'. We see that there a couple of types that need to be changed: Date and Zip Code, so we will try to do that now.

In [70]:
fully_clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2083674 entries, 0 to 1095688
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Store Number           int64  
 2   Store Name             object 
 3   Address                object 
 4   City                   object 
 5   Zip Code               object 
 6   Store Location         object 
 7   County Number          float64
 8   County                 object 
 9   Category               float64
 10  Category Name          object 
 11  Vendor Number          float64
 12  Vendor Name            object 
 13  Item Number            object 
 14  Item Description       object 
 15  Pack                   int64  
 16  Bottle Volume (ml)     int64  
 17  State Bottle Cost      float64
 18  State Bottle Retail    float64
 19  Bottles Sold           int64  
 20  Sale (Dollars)         float64
 21  Volume Sold (Liters)   float64
 22  Volume Sold (Gallons)  

First let's convert the date and zip code in our cleaned dataset to their appropriate types

In [71]:
fully_clean_data['Date'] = pd.to_datetime(fully_clean_data['Date'], errors='coerce')
fully_clean_data['Zip Code'] = fully_clean_data['Zip Code'].astype(int)

/tmp/ipykernel_1058/3355942340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fully_clean_data['Date'] = pd.to_datetime(fully_clean_data['Date'], errors='coerce')
/tmp/ipykernel_1058/3355942340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fully_clean_data['Zip Code'] = fully_clean_data['Zip Code'].astype(int)


Did our conversion work? Yes!

In [72]:
fully_clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2083674 entries, 0 to 1095688
Data columns (total 23 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date                   datetime64[ns]
 1   Store Number           int64         
 2   Store Name             object        
 3   Address                object        
 4   City                   object        
 5   Zip Code               int64         
 6   Store Location         object        
 7   County Number          float64       
 8   County                 object        
 9   Category               float64       
 10  Category Name          object        
 11  Vendor Number          float64       
 12  Vendor Name            object        
 13  Item Number            object        
 14  Item Description       object        
 15  Pack                   int64         
 16  Bottle Volume (ml)     int64         
 17  State Bottle Cost      float64       
 18  State Bottle Retail    floa

Now let's iterate through our yearly dataframes and attempt to convert Date and Zip Code columns to their appropriate types.

In [73]:
for name, df in dataframes_by_year.items():
    print(f"Processing {name}...")
    
    #attempt to convert 'Date' column to date type
    try:
        df['Date'] = pd.to_datetime(df['Date'])
        print(f"Conversion successful in {name}")
    except ValueError as e:
        print(f"Error converting 'Date' column in {name}: {e}")

#checking the data column types after conversion attempt
for name, df in dataframes_by_year.items():
    print(f"DataFrame: {name}")
    print(df['Date'].dtypes)

Processing Iowa_Liquor_Data_2012...
Conversion successful in Iowa_Liquor_Data_2012
Processing Iowa_Liquor_Data_2013...
Conversion successful in Iowa_Liquor_Data_2013
Processing Iowa_Liquor_Data_2014...
Conversion successful in Iowa_Liquor_Data_2014
Processing Iowa_Liquor_Data_2015...
Conversion successful in Iowa_Liquor_Data_2015
Processing Iowa_Liquor_Data_2016...
Conversion successful in Iowa_Liquor_Data_2016
Processing Iowa_Liquor_Data_2017...
Conversion successful in Iowa_Liquor_Data_2017
Processing Iowa_Liquor_Data_2018...
Conversion successful in Iowa_Liquor_Data_2018
Processing Iowa_Liquor_Data_2019...
Conversion successful in Iowa_Liquor_Data_2019
Processing Iowa_Liquor_Data_2020...
Conversion successful in Iowa_Liquor_Data_2020
Processing Iowa_Liquor_Data_2021...
Conversion successful in Iowa_Liquor_Data_2021
Processing Iowa_Liquor_Data_2022...
Conversion successful in Iowa_Liquor_Data_2022
Processing Iowa_Liquor_Data_2023...
Conversion successful in Iowa_Liquor_Data_2023
Data

In [74]:
for name, df in dataframes_by_year.items():
    print(f"Processing {name}...")
    
    #attempt to convert zip column to int
    try:
        df['Zip Code'] = pd.to_numeric(df['Zip Code'], errors='coerce')
        print(f"Conversion successful in {name}")
    except ValueError as e:
        print(f"Error converting Zip Code column in {name}: {e}")

#checking zip column types after conversion attempt
for name, df in dataframes_by_year.items():
    print(f"DataFrame: {name}")
    print(df['Zip Code'].dtypes)

Processing Iowa_Liquor_Data_2012...
Conversion successful in Iowa_Liquor_Data_2012
Processing Iowa_Liquor_Data_2013...
Conversion successful in Iowa_Liquor_Data_2013
Processing Iowa_Liquor_Data_2014...
Conversion successful in Iowa_Liquor_Data_2014
Processing Iowa_Liquor_Data_2015...
Conversion successful in Iowa_Liquor_Data_2015
Processing Iowa_Liquor_Data_2016...
Conversion successful in Iowa_Liquor_Data_2016
Processing Iowa_Liquor_Data_2017...
Conversion successful in Iowa_Liquor_Data_2017
Processing Iowa_Liquor_Data_2018...
Conversion successful in Iowa_Liquor_Data_2018
Processing Iowa_Liquor_Data_2019...
Conversion successful in Iowa_Liquor_Data_2019
Processing Iowa_Liquor_Data_2020...
Conversion successful in Iowa_Liquor_Data_2020
Processing Iowa_Liquor_Data_2021...
Conversion successful in Iowa_Liquor_Data_2021
Processing Iowa_Liquor_Data_2022...
Conversion successful in Iowa_Liquor_Data_2022
Processing Iowa_Liquor_Data_2023...
Conversion successful in Iowa_Liquor_Data_2023
Data

Now that our type conversions worked, we can combine the fully_clean_data back in with the yearly sales data. We do this by extracting the year from our dataframes list using a regex expression. Then we extract all the data in fully_clean_data corresponding to that year. Finally we concatenate this filtered yearly data back in. 

In [75]:
#iterate through dataframes_by_year
for name, df in dataframes_by_year.items():

    print(f"Length of {name} before additions: {len(df)}")
    #extract the year from the df's name
    year = re.findall(r'\d{4}', name)[0]  
    
    #filter fully_cleaned_data based on the year
    filtered_clean_data_yearly = fully_clean_data[fully_clean_data['Date'].dt.year == int(year)]
    
    #concatenate filtered_clean_data with the original DataFrame
    concatenated_df = pd.concat([df, filtered_clean_data_yearly])
    
    #update dataframes_by_year with the concatenated DataFrame
    dataframes_by_year[name] = concatenated_df

    print(f"Length of {name} after concatenation: {len(concatenated_df)}")


Length of Iowa_Liquor_Data_2012 before additions: 1884232
Length of Iowa_Liquor_Data_2012 after concatenation: 2041722
Length of Iowa_Liquor_Data_2013 before additions: 1872144
Length of Iowa_Liquor_Data_2013 after concatenation: 2032977
Length of Iowa_Liquor_Data_2014 before additions: 1909019
Length of Iowa_Liquor_Data_2014 after concatenation: 2075143
Length of Iowa_Liquor_Data_2015 before additions: 1984108
Length of Iowa_Liquor_Data_2015 after concatenation: 2162691
Length of Iowa_Liquor_Data_2016 before additions: 1962631
Length of Iowa_Liquor_Data_2016 after concatenation: 2185028
Length of Iowa_Liquor_Data_2017 before additions: 2028321
Length of Iowa_Liquor_Data_2017 after concatenation: 2228494
Length of Iowa_Liquor_Data_2018 before additions: 2129085
Length of Iowa_Liquor_Data_2018 after concatenation: 2340661
Length of Iowa_Liquor_Data_2019 before additions: 2158261
Length of Iowa_Liquor_Data_2019 after concatenation: 2363351
Length of Iowa_Liquor_Data_2020 before additions

/tmp/ipykernel_1058/1274342344.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concatenated_df = pd.concat([df, filtered_clean_data_yearly])


Let's sum up the number of rows in our dataframes to ensure no data loss. Then let's subtract this value from the length of our original liquor_data. Next, we will subtract the length of the_pile and the length of the Colorado Springs data we removed earlier(2 data points). If this result is 0, no data was lost.  

In [76]:
total_length = 0

#iterate through dataframes_by_year and sum their lengths
for df_name, df in dataframes_by_year.items():
    total_length += len(df)

# Display the total length
print(f"The sum of lengths of all dataframes is: {total_length}")

The sum of lengths of all dataframes is: 26982916


In [77]:
len(liquor_data) - total_length - len(the_pile) -2

0

We have finally completed the first round of preprocessing. Now we can simply save the yearly dataframes as their own csv files. It should be noted that I am declining to concatenate all the data together here as that has a tendency to crash the kernel. 

In [78]:
# Loop through dataframes_by_year and save each dataframe to a CSV file
for df_name, df in dataframes_by_year.items():
    file_name = f"{df_name}.csv"
    df.to_csv(file_name, index=False)
    print(f"Dataframe {df_name} has been saved to {file_name}")

Dataframe Iowa_Liquor_Data_2012 has been saved to Iowa_Liquor_Data_2012.csv
Dataframe Iowa_Liquor_Data_2013 has been saved to Iowa_Liquor_Data_2013.csv
Dataframe Iowa_Liquor_Data_2014 has been saved to Iowa_Liquor_Data_2014.csv
Dataframe Iowa_Liquor_Data_2015 has been saved to Iowa_Liquor_Data_2015.csv
Dataframe Iowa_Liquor_Data_2016 has been saved to Iowa_Liquor_Data_2016.csv
Dataframe Iowa_Liquor_Data_2017 has been saved to Iowa_Liquor_Data_2017.csv
Dataframe Iowa_Liquor_Data_2018 has been saved to Iowa_Liquor_Data_2018.csv
Dataframe Iowa_Liquor_Data_2019 has been saved to Iowa_Liquor_Data_2019.csv
Dataframe Iowa_Liquor_Data_2020 has been saved to Iowa_Liquor_Data_2020.csv
Dataframe Iowa_Liquor_Data_2021 has been saved to Iowa_Liquor_Data_2021.csv
Dataframe Iowa_Liquor_Data_2022 has been saved to Iowa_Liquor_Data_2022.csv
Dataframe Iowa_Liquor_Data_2023 has been saved to Iowa_Liquor_Data_2023.csv


The following blocks of code will help with the next stages of preprocessing. First, we need to get a list of the category names to later create boolean columns. We also output the_pile to a csv.

In [79]:
cat_data = liquor_data[['Category Name']].copy()
category_data = cat_data.value_counts()
#category_data.to_csv('categories.csv', sep= '\t')

In [80]:
the_pile.to_csv('the_pile.csv')

In [81]:
len(the_pile)

506825